<a href="https://colab.research.google.com/github/vuminhduc11/web/blob/main/LRFMP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install feature_engine
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from scipy import stats
from sklearn.preprocessing import StandardScaler
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans

import plotly.offline as pyoff
import plotly.graph_objs as go

import datetime as dt
import feature_engine

from feature_engine.outliers import Winsorizer

import warnings
warnings.filterwarnings("ignore")


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 357.1/357.1 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 45.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 78.9 MB/s eta 0:00:00
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.2.2
    Uninstalling scikit-learn-1.2.2:
      Successfully uninstalled scikit-learn-1.2.2
  Attempting uninstall: pandas
    Found existing installation: pandas 2.0.3
    Uninstalling pandas-2.0.3:
      Successfully uninstalled pandas-2.0.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pandas<2.2.2dev0,>=2.0, but you have pandas 2.2.2 which is incompatible.
google-colab 1.0.0 requires pandas==2.0.3, but you have pandas 2.2.2 which is incompatible.


In [2]:
df2 = pd.read_excel('Online Retail.xlsx')

In [3]:
df2['Price'] = df2['Quantity'] * df2['UnitPrice']

In [4]:
df2['date_diff'] = df2.groupby('CustomerID')['InvoiceDate'].diff().dt.days


In [5]:
df2['Moneytary'] = df2.groupby('CustomerID')['Price'].transform('sum')


In [6]:
df2['Length'] = df2['InvoiceDate']


In [7]:
df2.drop(df2[df2['date_diff'] == 0].index,inplace=True)


In [8]:
df2['Period'] = df2.groupby('CustomerID')['date_diff'].transform('std')


In [9]:
df2.head(5)


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Price,date_diff,Moneytary,Length,Period
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,15.30,NaN,5288.63,2010-12-01 08:26:00,NaN
9,536367,84879,ASSORTED COLOUR BIRD ORNAMENT,32,2010-12-01 08:34:00,1.69,13047.0,United Kingdom,54.08,NaN,3079.10,2010-12-01 08:34:00,18.162020
26,536370,22728,ALARM CLOCK BAKELIKE PINK,24,2010-12-01 08:45:00,3.75,12583.0,France,90.00,NaN,7187.34,2010-12-01 08:45:00,18.378176
46,536371,22086,PAPER CHAIN KIT 50'S CHRISTMAS,80,2010-12-01 09:00:00,2.55,13748.0,United Kingdom,204.00,NaN,948.25,2010-12-01 09:00:00,67.987744
65,536374,21258,VICTORIAN SEWING BOX LARGE,32,2010-12-01 09:09:00,10.95,15100.0,United Kingdom,350.40,NaN,635.10,2010-12-01 09:09:00,9.394147


In [10]:
df2.dropna(inplace=True)


In [12]:
Now = dt.datetime(2011,12,10)

In [13]:
lrfmpTable = df2.groupby('CustomerID').agg({'Length' : lambda x :(x.max() - x.min()).days,
                                            'InvoiceDate' : lambda x :(Now - x.max()).days,
                                            'InvoiceNo' : lambda x :len(x),
                                            'Moneytary' : lambda x :max(x),
                                            'Period': lambda x: max(x)})

In [15]:
lrfmpTable.rename(columns ={'InvoiceDate' : 'Recency',
                            'InvoiceNo' :'Frequency'},inplace = True)

In [16]:
lrfmpTable.reset_index(inplace=True)
lrfmpTable

,CustomerID,Length,Recency,Frequency,Moneytary,Period
0,12347.0,315,2,6,4310.00,18.478817
1,12348.0,243,75,3,1797.24,70.149840
2,12352.0,246,36,6,1545.41,68.419296
3,12356.0,222,22,2,2811.43,100.409163
4,12359.0,297,7,5,6245.53,40.276544
...,...,...,...,...,...,...
2117,18272.0,222,2,6,3064.78,21.830407
2118,18273.0,93,2,2,204.00,48.790368
2119,18282.0,114,7,2,176.60,77.781746
2120,18283.0,316,3,13,2094.88,17.625011


In [17]:
quantiles = lrfmpTable.quantile(q=[0.25,0.5,0.75])
quantiles = quantiles.to_dict()

In [18]:
def RPScore(x,p,d):
    if x <= d[p][0.25]:
        return 4
    elif x <= d[p][0.50]:
        return 3
    elif x <= d[p][0.75]:
        return 2
    else:
        return 1

def LFMScore(x,p,d):
    if x <= d[p][0.25]:
        return 1
    elif x <= d[p][0.50]:
        return 2
    elif x <= d[p][0.75]:
        return 3
    else:
        return 4


In [19]:
lrfmpTable['L_quartile'] = lrfmpTable['Length'].apply(RPScore, args=('Length',quantiles))
lrfmpTable['R_quartile'] = lrfmpTable['Recency'].apply(RPScore, args=('Recency',quantiles))
lrfmpTable['F_quartile'] = lrfmpTable['Frequency'].apply(LFMScore, args=('Frequency',quantiles))
lrfmpTable['M_quartile'] = lrfmpTable['Moneytary'].apply(LFMScore, args=('Moneytary',quantiles))
lrfmpTable['P_quartile'] = lrfmpTable['Period'].apply(LFMScore, args=('Period',quantiles))
lrfmpTable.head()

,CustomerID,Length,Recency,Frequency,Moneytary,Period,L_quartile,R_quartile,F_quartile,M_quartile,P_quartile
0,12347.0,315,2,6,4310.00,18.478817,1,4,3,4,2
1,12348.0,243,75,3,1797.24,70.149840,2,1,1,3,4
2,12352.0,246,36,6,1545.41,68.419296,2,2,3,3,4
3,12356.0,222,22,2,2811.43,100.409163,2,3,1,3,4
4,12359.0,297,7,5,6245.53,40.276544,1,4,3,4,3


In [22]:
lrfmpTable['LRFMP_segment'] = lrfmpTable.L_quartile.map(str) + lrfmpTable.R_quartile.map(str) + lrfmpTable.F_quartile.map(str) + lrfmpTable.M_quartile.map(str) + lrfmpTable.P_quartile.map(str)


In [23]:
lrfmpTable['LRFMP_Score'] = lrfmpTable[['L_quartile','R_quartile','F_quartile','M_quartile','P_quartile']].sum(axis=1)
lrfmpTable

,CustomerID,Length,Recency,Frequency,Moneytary,Period,L_quartile,R_quartile,F_quartile,M_quartile,P_quartile,RFM_segment,LRFMP_segment,LRFMP_Score
0,12347.0,315,2,6,4310.00,18.478817,1,4,3,4,2,14342,14342,14
1,12348.0,243,75,3,1797.24,70.149840,2,1,1,3,4,21134,21134,11
2,12352.0,246,36,6,1545.41,68.419296,2,2,3,3,4,22334,22334,14
3,12356.0,222,22,2,2811.43,100.409163,2,3,1,3,4,23134,23134,13
4,12359.0,297,7,5,6245.53,40.276544,1,4,3,4,3,14343,14343,15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2117,18272.0,222,2,6,3064.78,21.830407,2,4,3,4,2,24342,24342,15
2118,18273.0,93,2,2,204.00,48.790368,3,4,1,1,3,34113,34113,12
2119,18282.0,114,7,2,176.60,77.781746,3,4,1,1,4,34114,34114,13
2120,18283.0,316,3,13,2094.88,17.625011,1,4,4,3,2,14432,14432,14


In [24]:
lrfmpTable.drop('RFM_segment',axis =1, inplace=True)